<a href="https://colab.research.google.com/github/chicago007/test2/blob/main/%EC%83%81%EC%8A%B9%EC%A2%85%EB%AA%A9%EB%89%B4%EC%8A%A4%ED%8A%B8%EB%A0%8C%EB%93%9C%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install requests konlpy
!pip install Article
!pip install pykrx
!pip install pandas_market_calendars
!pip install feedparser

ERROR: Could not find a version that satisfies the requirement Article (from versions: none)
ERROR: No matching distribution found for Article


In [4]:
from pykrx import stock
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import requests
import re
import json
import pandas as pd
import pytz
import calendar
from IPython.display import Markdown, display
import feedparser

# 한국 시간(Asia/Seoul) 설정
seoul_tz = pytz.timezone("Asia/Seoul")

# 1) 최근 5거래일 중 20% 이상 상승한 종목의 상승률과 시가총액을 보여준다.
def get_krx_trading_range():
    krx = mcal.get_calendar('XKRX')
    today = datetime.now(seoul_tz).date()
    start_date = today - timedelta(days=30)
    trading_days = krx.valid_days(start_date, today)
    # 최근 5거래일 중 첫 번째(5일 전)와 마지막(최신 거래일)을 구함
    start_trading_day = trading_days[-5]
    last_trading_day = trading_days[-1]
    return start_trading_day.date(), last_trading_day.date()

start_date, end_date = get_krx_trading_range()
print(f"시작일: {start_date}, 종료일: {end_date}")

# 주가 변동 데이터 가져오기
df = stock.get_market_price_change(start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"), "ALL")

# 등락률이 20% 이상인 종목 필터링 및 내림차순 정렬
rising_stocks = df[df['등락률'] >= 20].sort_values('등락률', ascending=False)

# 시가총액 정보 가져오기
market_cap = stock.get_market_cap(end_date.strftime("%Y%m%d"))

# 결과 출력 및 종목명 리스트 생성 (종목명(티커) 형식)
print("최근 5일간 20% 이상 상승한 종목:")
stock_names = []
name_to_ticker = {}  # 종목명과 티커 매핑
for ticker in rising_stocks.index:
    stock_name = stock.get_market_ticker_name(ticker)
    name_to_ticker[stock_name] = ticker
    change_rate = rising_stocks.loc[ticker, '등락률']
    cap = market_cap.loc[ticker, '시가총액']
    cap_billion = cap / 100_000_000  # 억원 단위로 변환
    print(f"{stock_name}({ticker}) (등락률: {change_rate:.2f}%, 시가총액: {cap_billion:.2f}억원)")
    stock_names.append(stock_name)

# 네이버 API 정보 (발급받은 값 입력)
CLIENT_ID = "DjAgMkN04y03NqmM6SpU"
CLIENT_SECRET = "y5vuriJA8E"

# 날짜 설정 (트렌드 데이터: 최근 30일)
END_DATE = datetime.now(seoul_tz)
START_DATE = END_DATE - timedelta(days=30)

def get_trend_data(keywords):
    url = "https://openapi.naver.com/v1/datalab/search"
    headers = {
        "X-Naver-Client-Id": CLIENT_ID,
        "X-Naver-Client-Secret": CLIENT_SECRET,
        "Content-Type": "application/json"
    }
    # 최대 5개씩 나누어 API 호출
    chunk_size = 5
    keyword_chunks = [keywords[i:i + chunk_size] for i in range(0, len(keywords), chunk_size)]
    all_results = []
    for chunk in keyword_chunks:
        body = {
            "startDate": START_DATE.strftime("%Y-%m-%d"),
            "endDate": END_DATE.strftime("%Y-%m-%d"),
            "timeUnit": "date",
            "keywordGroups": [{"groupName": keyword, "keywords": [keyword]} for keyword in chunk]
        }
        response = requests.post(url, headers=headers, data=json.dumps(body))
        data = json.loads(response.text)
        if 'results' in data:
            all_results.extend(data["results"])
        else:
            print("Error: 'results' key not found in API response for chunk:", chunk)
            print("API Response:", response.text)
    return all_results

def analyze_trends(trend_data):
    df_list = []
    for item in trend_data:
        df = pd.DataFrame(item["data"])
        df["keyword"] = item["title"]
        df_list.append(df)
    if not df_list:
        print("Error: No data to analyze. Check the API response and parameters.")
        return pd.DataFrame()
    df_combined = pd.concat(df_list)
    df_pivot = df_combined.pivot(index="period", columns="keyword", values="ratio")
    # 최근 7일과 이전 7일의 평균 검색량 비교
    recent_7days = df_pivot.iloc[-7:].mean()
    previous_7days = df_pivot.iloc[-14:-7].mean()
    growth_rate = (recent_7days - previous_7days) / previous_7days * 100
    return growth_rate.sort_values(ascending=False)

# 트렌드 데이터 가져오기 및 분석
trend_data = get_trend_data(stock_names)
growth_rates = analyze_trends(trend_data)

print("\n검색량 증가율 (%):")
for keyword, rate in growth_rates.items():
    ticker = name_to_ticker.get(keyword, "N/A")
    print(f"{keyword}({ticker}): {rate:.2f}%")

# 상승종목의 뉴스 가져오기 (최근 7일)
news_end_date = datetime.now(seoul_tz)
news_start_date = news_end_date - timedelta(days=7)

headers_news = {
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret": CLIENT_SECRET
}

def clean_text(text):
    text = text.replace("&amp;", "&")
    text = text.replace("quot;", '')
    text = re.sub(r'<.*?>', '', text)  # HTML 태그 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수 문자 제거
    return text

for query in stock_names:
    ticker = name_to_ticker.get(query, "N/A")
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&sort=date&display=10"
    response = requests.get(url, headers=headers_news)
    if response.status_code == 200:
        news_data = response.json()
        print(f"\n최근 7일간 '{query}({ticker})' 관련 뉴스:")
        count = 0
        for item in news_data["items"]:
            # pubDate를 %z 포맷을 사용해 파싱 후 서울 시간으로 변환
            pub_date = datetime.strptime(item["pubDate"], "%a, %d %b %Y %H:%M:%S %z")
            pub_date = pub_date.astimezone(seoul_tz)
            if news_start_date <= pub_date <= news_end_date:
                count += 1
                title = clean_text(item["title"])
                link = item["link"]
                print(f"{count}. {pub_date.strftime('%Y-%m-%d')} [{title}]({link})")
        if count == 0:
            print("최근 7일간 관련 뉴스가 없습니다.")
    else:
        print("네이버 API 호출 실패:", response.status_code)

for query in stock_names:
    ticker = name_to_ticker.get(query, "N/A")
    # 구글 뉴스 RSS 피드 URL (검색어: 한국무브넥스)
    rss_url = f"https://news.google.com/rss/search?q={query}&{ticker}&hl=ko&gl=KR&ceid=KR:ko"

    # RSS 피드 파싱
    feed = feedparser.parse(rss_url)

    # 현재 시간과 7일 전 날짜 계산 (한국 시간 기준)
    now = datetime.now(seoul_tz)
    cutoff = now - timedelta(days=7)

    news_items = []

    # RSS 피드의 각 항목을 확인하여 지난 7일 이내의 뉴스만 선택 (최대 10개)
    for entry in feed.entries:
        # published_parsed는 time.struct_time 형식이므로 calendar.timegm을 사용해 timestamp로 변환
        ts = calendar.timegm(entry.published_parsed)
        published_dt = datetime.fromtimestamp(ts, seoul_tz)
        if published_dt >= cutoff:
            news_items.append((entry.title, entry.link))
        if len(news_items) >= 10:
            break

    # # 결과 출력 (헤드라인과 링크)
    # print(f"\n최근 7일간 '{query}({ticker})' 관련 뉴스:")
    # for idx, (title, link) in enumerate(news_items, start=1):
    #     print(f"{idx}. {title}\n   링크: {link}")
    # 결과를 Markdown 형식으로 생성하여 출력 (제목 클릭 시 링크로 연결, 링크는 노출되지 않음)
    markdown_text = f"\n최근 7일간 '{query}({ticker})' 관련 뉴스: (최대 10개):\n\n"
    for idx, (title, link) in enumerate(news_items, start=1):
        markdown_text += f"{idx}. [{title}]({link})\n\n"

    display(Markdown(markdown_text))


/usr/local/lib/python3.11/dist-packages/pandas_market_calendars/market_calendar.py:179: UserWarning: ['break_start', 'break_end'] are discontinued, the dictionary `.discontinued_market_times` has the dates on which these were discontinued. The times as of those dates are incorrect, use .remove_time(market_time) to ignore a market_time.
  warnings.warn(


시작일: 2025-01-31, 종료일: 2025-02-06


/usr/local/lib/python3.11/dist-packages/pykrx/website/krx/market/wrap.py:188: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace('', 0)


최근 5일간 20% 이상 상승한 종목:
에스에이티이엔지(351320) (등락률: 101.27%, 시가총액: 0.00억원)
바이브컴퍼니(301300) (등락률: 65.93%, 시가총액: 0.00억원)
플리토(300080) (등락률: 62.93%, 시가총액: 0.00억원)
GRT(900290) (등락률: 55.73%, 시가총액: 0.00억원)
진양화학(051630) (등락률: 53.33%, 시가총액: 0.00억원)
레인보우로보틱스(277810) (등락률: 50.20%, 시가총액: 0.00억원)
에이원알폼(234070) (등락률: 49.40%, 시가총액: 0.00억원)
엔솔바이오사이언스(140610) (등락률: 47.77%, 시가총액: 0.00억원)
루트락(253610) (등락률: 46.75%, 시가총액: 0.00억원)
한국무브넥스(010100) (등락률: 44.86%, 시가총액: 0.00억원)
SOOP(067160) (등락률: 44.50%, 시가총액: 0.00억원)
카페24(042000) (등락률: 44.47%, 시가총액: 0.00억원)
솔트룩스(304100) (등락률: 41.67%, 시가총액: 0.00억원)
토마토시스템(393210) (등락률: 39.17%, 시가총액: 0.00억원)
피델릭스(032580) (등락률: 39.16%, 시가총액: 0.00억원)
인스웨이브시스템즈(450520) (등락률: 38.73%, 시가총액: 0.00억원)
코난테크놀로지(402030) (등락률: 38.39%, 시가총액: 0.00억원)
온코크로스(382150) (등락률: 37.24%, 시가총액: 0.00억원)
엠케이전자(033160) (등락률: 36.70%, 시가총액: 0.00억원)
클로봇(466100) (등락률: 36.26%, 시가총액: 0.00억원)
디와이디(219550) (등락률: 35.97%, 시가총액: 0.00억원)
라온테크(232680) (등락률: 34.76%, 시가총액: 0.00억원)
아남전자(008700) (등락률: 34.69%, 시가총액: 0.00억원)
유니온(0009


최근 7일간 '에스에이티이엔지(351320)' 관련 뉴스: (최대 10개):

1. [[급등락주 짚어보기] 에스에이티이엔지, 최대주주 변경 기대에 이틀 연속 上 - 이투데이](https://news.google.com/rss/articles/CBMiVEFVX3lxTE91RGoxUndLdWJpNmhQaEZrNzJMZjBZQnFWODBxYzQ5VVd5LUVfejZGcm51SlZhaUZfZUtLUmJUOF9sSGtuZl96ZXl4YXBqZG9aVi1xQw?oc=5)

2. [에스에이티이엔지, 상한가 직행 배경은? - 영남일보](https://news.google.com/rss/articles/CBMia0FVX3lxTE5jS0M5Z1RoMWQ1S0hCbzZwbXRvcEx4bTYzd2NIYmRwQXV1QzNvaUFsSElyYlFjSUV2OWh4c0h3YUE3RXdEWDdFOVpuTGFBSlNaQnIybHJYMTR2enRpY1NNc1VFa2VRV1hPZmlj?oc=5)

3. [[에스에이티이엔지 M&A] 나노캠텍 ‘주가조작 세력’의 귀환① - 녹색경제신문](https://news.google.com/rss/articles/CBMiaEFVX3lxTE8weDJBazdBQnhSYktQdV9ySU5rSXM1el8zNFh2SWZBdlpzakRpSUlsTXAyNGRBcXZsRUtaS0F6RWFvV0U3X2VzS2pHRk1KMVBEd3RXLU9Yb0R6ZExFUzNhNi1XdkpNSWRQ?oc=5)

4. [[특징주] 에스에이티이엔지, 부동산개발업체에 매각… ‘上’ - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMihwFBVV95cUxPZHZOWEFBbi14bmNqMGVObWk2d0wwZFJDVHdDUUhTM3F2SXZ0bkt5RGlSR0t1a2FCcVVieWg0RkFmbGxDc2cydlJOQzhKbGlaVW80MVNPM0Z5VGc5VHhucmJPYm04SVQ1QlE5N0tCQVFJU2k4WFA1RGc5aC1rZElLd3hfQUtVLUnSAZsBQVVfeXFMT3ZNbXBzRmJxa3VGMDZuN2JIOEl0Nk5ZSm9UcDJsM0JUdHpCSTFZSWxBcFdKS0NvVlBKbUVDRnJXOS1XVHhvc2dDa0ZTcV9peTJpSk9QcThWcmo5NU5TbHVXbWVyd0ExUVZ2VGlEY3FKYVlmcWxfc2J2T2pHMnVTN0VEUDdySHlUUVdJUmZVT3k4bm9BbTF5RnNvSG8?oc=5)

5. [[급등주] 에스에이티이엔지, +30.00% 상한가 기록 - 전국매일신문](https://news.google.com/rss/articles/CBMibkFVX3lxTE8tNGhLQWN6TXJSa2FhQXdBWEwtOVM5NEcwdVdWRFZmMzNHS1BESlJ6cDNPRDhlZnppWmQzcnFNYllXSGQ1b2Y4RzdhWXZSWVpwQmd4Sjd0YWJsUmpSeTlXRVdaWVdWUjRrdzdrTlVB?oc=5)

6. [[특징주] 에스에이티이엔지, 최대주주 변경 소식에 '상한가' - 서울와이어](https://news.google.com/rss/articles/CBMibEFVX3lxTE5tT1BBOGxldElRb3B5Qi1UdnU2aWprS2RuZzRKZy1NY2FsOHZVSEczeGhXUkxiaXhscHFxVFBmLXZnNk5GTEVpY1NMQy1RVXpQYjR2dlVDV2VycUtnSnh1czlXYVd4M3Jtc0JkLQ?oc=5)

7. [[급등락주 짚어보기] 에스에이티이엔지, 최대주주 변경 기대에 이틀 연속 上 - MSN](https://news.google.com/rss/articles/CBMi5gRBVV95cUxQV1hfLXRORUJib1U1OU4waVBSRDNGVzZHY2ZYQ0ZiTXNBSTdXRU1jeUF1UjNwSU54cEFJVU5KYWhGQU5EYkdBc3RsR1UtekJJRE1TQXlCVkU3dkl3OUtMWDNMS1N2NHRpd0RSQ0hzSEJfcnBfUU9fay1STFZtcXFjNTU3WTVaN0dBZmloMzdkT1lnUVhNeDFMMGdxUW9aSUdKZWYzWlF3SWctVDQ4aDBzMjE3RkZjeFhlQnlHY3FOc2hxNDlWekhLNFQzb2xRZWxsRVNnb1Z6MlZRemstMmh6T0ZrNzhmMEUwMW44Yk1HWVRPZDJBSG1nMGszNGlycVR6QVJqVnRray1tR2hiUHNoNkEwQ09JZHlOMlI3VXdMbWpuYmZVU3VmZE1SQS05Q0ZyTDEyanNCQ3A2cEdqRnFNMmM0UGw3UUYtRng2cy1SNkdUUVF2dUR0UU5zR01vS3pWLTE0S25ya193eVoyeUxsUG14Q2FIWHVJbTRVOFcxR2ttRXU2YkdpUEhjVEY5U2FmcGhqQWk3Y3l3SmhPMGd5VVpoRjRWTjY4WUptLXdlS3Nod0JSU2JySEZLckk1SmV3RVJiSW9rSTJKWU5vQ0EtcnZ4TlhUQWxTU2lDc19HanJ3U2hiSnU2MHlQRE5CZDZhWWdKSjhjWTVLWEhrYndIUlhCT0cycWFOR25nQnpsRzVmTUlYLUJHSWxpV1BJUkpKOWpkWXNLTzh3OXIzT2NhY3NFVGYxSE5mQTdIOWRTV3M1UGVPalJjQ200Z292VnhhNWsxYnZmUkxCZTVZUTNMWUhlcWtiZw?oc=5)

8. [에스에이티이엔지, 300억 규모 전환사채 발행 결정 - 이데일리](https://news.google.com/rss/articles/CBMigAFBVV95cUxOdmJyMFBwNXFtZHREU200bTVTV3NhSFMxZk9TZEhnZkZ3ajRWX01OZHZfbTFrV2Nxa2hJVFZrdmpuWGl2RjVqYzRpMDBfbVNpejVudU9RV3dLZW5aem5ONHpIV1RSb1RudHM1X3g0RjlJNXlQWUc0VG1qNURVMWY2Tw?oc=5)

9. [에스에이티이엔지, 자금조달·최대주주 변경 기대감에 2연속 上 - 동양뉴스](https://news.google.com/rss/articles/CBMiYEFVX3lxTE5JMFVhOFdPdll6OW4xVXh2OTdUaGdlVzJmQWRSTkQtczgtOHJiRkZ1Zng4UDh3Ny0tRHZYUS05NkduUkVVa29pWVNmbC0xWVN2MU4wcElfaE0xY0o0cEswdQ?oc=5)

10. ['300억 규모 전환사채 발행' 에스에이티이엔지 주가 29% 상한가 - 국제뉴스](https://news.google.com/rss/articles/CBMibkFVX3lxTE5GUlk3SmlFd1lVSXJXTGdvaTlUOGM5ZFZlVlFwaWQ0LUZocl9aVHM1eEFWRTd0SlZoSXB0Q3FoYkZFbWVpZDRkc01hMUtCVl9SdGlialpuUzRzNXlhVlZidTFDdV9pSWY0Wm1ybFN3?oc=5)



...


최근 7일간 '아이케이세미콘(149010)' 관련 뉴스: (최대 10개):




최근 7일간 '코어라인소프트(384470)' 관련 뉴스: (최대 10개):

1. [코어라인소프트, AI 기술 기반 글로벌 원격진단 시장 공략 속도 - 뉴스핌](https://news.google.com/rss/articles/CBMiXEFVX3lxTFBCR0g4cXh6bWd6WE1SdUpCckhxb3FmWS1mZVd1eEFkV2Qxckl1c2pMdHdFRWVPWTBoclZ2dUFwR0ExcG4xdlFLbTg2aHp1VTRad2lhXzVKWlVtLS010gFeQVVfeXFMTndSNnRReU9EVVlqa0IxdW05V1dyYUoxU0V3cHVLS2JFWVhjUDNTR1FMUC1Yei1mQkQzWjlLTDVaSGNyenFjemM2Z0plSEh1OW5hYk9uWGtNVXhsZ3Nfdw?oc=5)

2. [[생생한 주식쇼 생쇼] AI 의료·신약 개발 강세…코어라인소프트 (384470) 신규 편입 - 매일경제TV](https://news.google.com/rss/articles/CBMiaEFVX3lxTE9Bd241eEptbTVTUVkyUVQxNUZJVmMwMGRsa1hmdmtFbl96OEx0bldUR0cwRDdhR2JLelEyVEJHQ0JfZEVscEdwVFhFaEY1MG9mTHpGeWo2WUt5YnpqNnJ3c2ZBRUpzOWli?oc=5)

3. [코어라인소프트, AI 원격진단 시장 공략 가속화 - AI타임스](https://news.google.com/rss/articles/CBMiakFVX3lxTE5TQktNS2JYQ25rTWVLdWJ1QW84N3owQjJ6eTc0d0E3YmdfX1B2ODhDZTVlR05PX0dTVE42RjJSd3FaTjg5QnZiMHFyblVVQkVhdHpRTFFidWdyR1RnN1lsYnUyU0tKLVVueWc?oc=5)

4. [코어라인소프트, +1.65% 상승폭 확대 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxPTEtCM3NrR05pTDBIcktvbmpQLVMwSldjczEwTzdDREdmcEJHMnptVXFqV0J3SFIwejBkeW9HYU56SFRObFl3SUZGR3kwMTh4dkpKc2d1c0t1Q3BhbGsxQ0swaVJiMUJDRlZVcDNEZjNrRk8zWmlmTEVhc01kLUxuM2FCd9IBlwFBVV95cUxPZmZONm1HR29hSHBhWXJ6bEctV1FQNDFPRDNKR0Y3VFM5azFIb1pta09DX0FSSHdyRXNrS0NMRERjV3U0SzhCb3FZYm0wclhCRFpWVU9IM0wzd3NVNC1IdEhkUU9USUhsUDRheTlUUk5vYzVFNWhHbEhUWUdLQ25KNWxLM24ya2lzMjJvTGpoeU91Sy1ocUMw?oc=5)

5. [코어라인소프트 “글로벌 원격진단 시장 공략 가속화” - 라포르시안](https://news.google.com/rss/articles/CBMibkFVX3lxTE5WVlkyQjd5S1ZDTU1ZcG44ZTh4RmVTV1VORW53XzdoM1lrSHhkTy1TM2tLTURQeno5RXV6WU9CNUV4Y1RfRzN2VTVCREhiYm10aEVKVHpKcGJoT1V0ZUFhRUNZRWFqSUhPU2JDRzZB?oc=5)

6. [코어라인소프트, 원격진단 시장 공략 가속화 - 의학신문](https://news.google.com/rss/articles/CBMiaEFVX3lxTFB6cnNwZXNHLXh3NGZudE9sRnFrWVcwUTRDclFyYUZsTkxIVnhuT25sRWhwWVR2M1lQVnNBTEZYTFhESGZRY2o4YnVmUkx0Uk0zM1ZqbW80bGUxX3lJWTBUblUtZ1pscHN2?oc=5)

7. [[단독]김진국 코어라인소프트 대표 "호주 1위 헬스케어社와 협업...글로벌 시장 공략" - 팜이데일리](https://news.google.com/rss/articles/CBMibkFVX3lxTE1kUV85Y1c3ZUNrNWJCU19jbS1GY05UM3JBNVlETENHajhTdXJwb1RtMV9acTR0RmhDbzU2R1BjaWxiZmt5Nzc5elJ1RGhmY0s1U1VJdVprWmlrdm93RUtMcV9aY2o1RmZfYjBsNlRR?oc=5)

8. [코어라인소프트, AI 기반 글로벌 원격진단 시장 공략 박차 - 더에이아이(THE AI)](https://news.google.com/rss/articles/CBMiakFVX3lxTE02QWtuQjJ2MWpXRC15X1RhaUVnc3BSaS0xNWU0NWU3UWhILVpBbGs2LXhzcm45emp1WUY2ZGpjQzJHTXBFbGRscUJmaDdmNWJvRmdQQXVSMS1aMm1yMVE0LWxOV0tQUjVLRXfSAW5BVV95cUxNXzM0NnR0SG5FYWN6VkJNTElQa1g3YjNxRXY1NFJtUUFpZjlxTHRwRlJFc2dkTnNSRzk3WERzeU5QdFBiYzRpWUN6SFVWdmpXejF2V3NoS3pWanJ4V0dYQmVWVFZJNmNuaktCX1gwdw?oc=5)

9. [코어라인소프트, AI 원격진단 세계 정복 나서다- FDA, CE 등 해외 인증 다수 보유 - Korea IT Times](https://news.google.com/rss/articles/CBMicEFVX3lxTE5oSHgwTXJBOXBTVzlaVWpSN0VLTVowN2d5Y2tMMmFZdGFEQ2gwSXpUX2Ewd0cxWWplQzM5aXdQRzdJMTdsSVJldWpSMU41X0VIWTVjWDhJbTNEYWVZLWM4eXVQNjlZZlRxcG9fY0dfRUQ?oc=5)

10. [코어라인소프트, AI기술 기반 글로벌 원격진단 시장 공략 - 동양뉴스](https://news.google.com/rss/articles/CBMiYEFVX3lxTE9McmpyLUJWZlpBQ1o1dW1pZm9rbnQ0SnVvaEJGckNqbFlLV0NWZGRCalhCMEdNZkYzbDR4NEdkRXN2MzRzYUpsNU4wOW9meFM1bVdvTlBLX3hxZDNXczloUQ?oc=5)




최근 7일간 '디티앤씨알오(383930)' 관련 뉴스: (최대 10개):

1. [디티앤씨알오, 올해 약동력학센터 가동 눈앞…글로벌 시장 공략 속도 - 네이트 뉴스](https://news.google.com/rss/articles/CBMiU0FVX3lxTE00TVJocy1Hem54WDJYM2xJUWRHcmNrLVN2blZWdFpYZDFyNWFmVVQxUmhzQXF2OWRhSU4zZHM2bUJSMGlZX0x4eHNHS05DUGFTd1Vn?oc=5)

2. [[강세 토픽] CRO(임상시험수탁기관) 테마, 디티앤씨알오 +5.78%, 현대ADM +5.65% - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxQQjkzLXJvalhCMC1rSTlkQWpXTmxoVW4tTlF3N2I0bVRjZmhERmNTa1hZdFVTM3hPdEZyRHg2UnJISXpiZ1NwU09vMnZZTlhycldWbXZzWF94VEZSQ2I0MlRSN1NucUZickl6Q0t0QmV3YlN1UmRHWjFQWXBxX2g2bWx5Z9IBlwFBVV95cUxQcTZpclVBT0lsR3pPNW5uUEphQkZQYXNVTUhzQi1WSFBzZUR2ZUtrcks0cXpJU0dMVWVCd3JxTXBiTWJxcE1nYzZZLVhiSy1PZk9jeU1EaF9KbzFNMjhFY2pEQ1R0V1UydUFJUW9SMFpqZ0Q0VTdyY2xyb2tBdGs2U0tYZ2hQZFU1MFBDUm80a1BFNXh6YVQw?oc=5)

3. [디티앤씨알오, 올해 약동력학센터 가동 눈앞…글로벌 시장 공략 속도 - 머니투데이](https://news.google.com/rss/articles/CBMibkFVX3lxTE0tYTNra1F0R1FVUG81N204UkI5Z2ZMek9rX196aGQtREZfTEpJWmg1ODhzT2h5YkhpX2VLSzNQT1dLSm9aQXF1dk52MFJSb3JJUDQ5c3E0MFVUYmhwUGdiUm1aYV9SdDNsRklzZHdR0gFuQVVfeXFMTlpYSnllUnlRSkNJNmRzQ0JyOWZBVVQ2X0xSeGNDdk5yREZBbnVrWFl1SDFtRF9XNUp6WEVRUFdNZ0Y0bjJVbzNDakZ3UGEyVFVSTGZFb2ZNOWQxZ0NKWjRiZW83QnV5bFo3a2s0TGc?oc=5)

4. [디티앤씨알오, +11.31% VI 발동 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxPLWlELWpVb0lYcjFOOWNlVWF6NFZjSVg3elQzd1BvS1U4LTA5ekczQjBMTi02ZXRLazVldW91ZGFINmdaaGZWV1YzWnlqV2VVQVRmM05NaFk0clVhWFlWdWczaEVhU0MzUThvbHB2Mmk1SEU1WDFJV1VVM1YzM01SQXd0Z9IBlwFBVV95cUxOb1pGWEhYSXhmeHNqOXN1Y1pxTnNnQnlFcGYySUpkU2xsdEhuRzB5MU5kQW5iYVNfZzF5bmNHTnRyd1ZLNG5WQ25OajBKM3BRWEpWOUlDc1Vlem1fd0Y5Rk9rUm9ua1JSdVI0aDZlUTlGWFhkTV9JaF9tWWJvaDlYcUFQNXl6TlExb0M3MHBfaXVVd3lrWTln?oc=5)

5. [[특징주] 디티앤씨알오 주가 급등, '尹 집행정지 찬성' 발언에 한동훈 테마주 부각 - MSN](https://news.google.com/rss/articles/CBMi9ARBVV95cUxNTzFlNWt0Z3BOVUVhaTNlXy1EQ3VuS3BsNWZVS2QyaG4zeVV0UGhqdWhkTnV3UWtvYUZlY0hBeVJqbU1ZR1ZDZGl0RXlwQnFhMjdVdlY0Z1hQbWVGV3V2ZVBoR0dQMWRPYW9oY2lkbDRnQUZFdVRYVHRYNERYSm1STmk3cm8tTVV1TUYzYVhUcU1VbkRDbmRxWmVBQTVIQjV6Rkd1WDN4TVFQblpVVENkQlRjdGZXeElkM292bEtobkZrVnMxRnVEa1ZqbURpVHBmWEticzUtTFdKdGxMcEdXMXRuQTRIdVJIOHV1RWd5NDJxZV9YU1dxVkNvV1NtNm11M1NxQ2JKcjV5RkhVMEFwQk1zeGpaWkxlekp1eWxqRmxxVXFWdk9ZbUlLVGdmc0p3VUd0YkVqWU4ybVRCOUtiM1ROMV9MelFzSzRhUEhsamc1NmozX0MwUkh3V1pkZGRvcmN1Z01BSXdZNHhZNmo2M18zZVNlOTJQUmNhNEdkRXByQU9heV9rcmNac09LdkVNS2FfdHpEWF9nY3V1Q3lJd1BoM0hGbVR6NHkyb1JEeXluZE8zbWZFMFFNQ1RTVWl1SVRuN2VBNWN2cENiZWd5NURDdFU1bDlHOVB6MWV3N3FUZVE0b0VfQzFtVnhPcmxEY19rR0xMNFVUV0pTYVI3M1FIal9sdkRMRy1qSWZjR3JWODRFYlpQMGpkMF8ySmtQbzVnalMzbm5pWDhqRlB3c0dwRHZYVjc4NWhadUI0Q3FITXp0ODk5LXRsZWw0QUZ3Y3VueGM1Vm11aEdmY0tPTnQ2bXVzQm4zdnZxTnVaX1lZZXYx?oc=5)




최근 7일간 '브레인즈컴퍼니(099390)' 관련 뉴스: (최대 10개):

1. [브레인즈컴퍼니, +12.01% VI 발동 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxNLWtaUUJWVy1aWXlzaUk5MzEwd0FDS3hlV1M4R0FzNlBsS1JSMVZLNURkcUZmb3c5M1F5UGduVEFiM0hnRVlabXpTYnIwNlBidUJWb3J5TmFVODZrb2ZORmh0WVlWT010T25Zc2ptVHBlQWRBOTUtYTctM19MNE4yb200VdIBlwFBVV95cUxQS3M4eFlGWlhUa1JyV1VmQUE0Nk96RjMweloxQVUzSlJEWUdlbGZvSlZBdkU3cFJhbElSM1ZSTmRuNTR6RVpJSG16MDhPbnRjd1ZGbzFOMnE1ZGtHUjZOMUxUejRydEVpOE9LNjU1UGdDNUNHa1FzNVFlZHlIWmtRYjN5X0JHVjdtWUhoNzk2UTR0c1BhV3VN?oc=5)

2. [브레인즈컴퍼니, 주가와 거래량 모두 강세... 최근 5거래일 최고 거래량 돌파 - 조세금융신문](https://news.google.com/rss/articles/CBMiY0FVX3lxTE5NdFBSY1hGemZfWUo3clUtQ1pteVMxcXl3YzVpSk0xdFZQVzZTUnZyeXExeTdZa0VlQTNzUFFkaF9lZlVvdDd3YzNyUkdsdE5jR01fUHZnY21BMUtPOGJMZlZ6cw?oc=5)

3. [특징주, 브레인즈컴퍼니-AI 챗봇(챗GPT 등) 테마 상승세에 5.35% ↑ - 매일경제](https://news.google.com/rss/articles/CBMiUkFVX3lxTE05Rk5LclMwLUl4b2NMU1hVNnR0ZmcxXzBFVEV3WTAxUlZpQ0hmb29ZNGMzRVpuRm0zZXhlazF0ZTRmXzNKNWlOWk9OREZkY2tJemc?oc=5)

4. [브레인즈컴퍼니, +9.01% 상승폭 확대 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxPS09xZFFOMEhmY3ZFdklCblZUV1pVY016QWRBSjJScHRZZDFxREI4dE1YZzBWWnJUYkFOMUhfQnNwZnpFSDc1dFV6MWl3WlRiTzE5Ni1pZzRqR3JmX05iM0RpWm9ZcEM5U1BLUTJsQXFNWnd3MkdzaF85OTlDNWVEYXdETdIBlwFBVV95cUxOZFN0U3c3RU1PUkRFZmtRYnpEN2RmT1hDVFAtZmE5cWozRkZ2cnNJcHczdUcwQXYtaGtqdjk2NXI1a1NvbGNrMXBUN2Q2aUhrTnlMazFBQVg1Z2JGNGUzME10NjBOQXp6MUdOYUZlbUhnMDd5dDg4eWdlRFhQQkVBWGJ2QXZ5N0ZLTmF5cWs3ano3Z3lIYkdj?oc=5)

5. [특징주, 브레인즈컴퍼니-AI 챗봇(챗GPT 등) 테마 상승세에 6.89% ↑ - MK빌리어드뉴스](https://news.google.com/rss/articles/CBMiU0FVX3lxTE9Qc0c4SklTYkhCRXBCY3pNcUJ1S2JLZFNOZnA4OUhjMEdQa0xIMUxFLXVrRVp0OTBOSExkdDN1WXkzaklTczNIVFdNUVVzWWZCVS13?oc=5)

6. [브레인즈컴퍼니, +11.21% VI 발동 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxQRHBMUFFzc1JqR09aQ18tOUh4ZnB1aW5SdDVzVmlTeWY2MV9xZVJCd3lqTkMyTl9yZ0U5RjhJN3d3VkZ6U1doYk0yLVhfUFBuUnJPN1d5cjM5LTNiUU1oU3JiZVZrUmREN1pLZTFYcFAwNEh5bDRrbF8zVTlwbUFNODZETdIBlwFBVV95cUxPR0lKMFZ4YVRzN3c0RDhYdm0wU01FYkdkNUc2X0xUbFBkY0toNVp3LUN6WFVnNlJ1ZjVCZXBBblMwaHpjbllTNnRnaTZSelNmWDdUa3JXajdvMW5BbWNmWWJlWFB1dFY5Vnd5eFotMG5neGtfVzRGNW5pNUxObnZ4UnhUUzJOaWFFbWl4MmNxUEh4QS1zM1dF?oc=5)

7. [브레인즈컴퍼니, +6.99% 상승폭 확대 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxNX21WT3EyZ3BIaGhYVm5oZFVpcGFfa1VZdXdDczNCV0V4V0JyUnVMYnNGdjJhRXhfU1BYQWtVQXk1b3FLRjhBOGRCQzNiYld3c2FPenNvQlJCUXlRNmFKWXF2VDNBbGdXU1lfVmNVbXNJVzNaLVhKMzh2bEZIWGlCVFVEZ9IBlwFBVV95cUxOc19hYS1iY3pGbDhZZGJKY2NuOHBERWtEM1FpQXFtcjdUVnc0bmd1OC1QVzR5d1NyeXh5bnhtZmVlcUdPYUVobGRxTFVxM0hlNkV5VEpYb3ZlZVRIbXYyazRlRUY0bnNnaFJGQVYzVWhTYlJqZHJaRmpKRHNDal9nNGNQTkI2Njd5eEFybzZ2NTk2UnR0NlNJ?oc=5)

8. [브레인즈컴퍼니, +9.90% VI 발동 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxQczBGUi12Ynk0Y0xEVnlwaUE5Y05tRzRaVnFEN0E1b25JUmJCS0xJb3hjM2lSaEh3THdGOU94Tl9jVGZFNndSdVI1d0lkdS1GUTFDSkZFekFSWlZQYXp0SkMzbVZyR0pIVDdBOUdzNXVYcnBiR3RNMFROeGctSVNJbVdNVdIBlwFBVV95cUxOMDdSWnhtdlFKOWM0NVZJSWhQTklCS1UwdllxcjVOYkpOdkl0UzJjVkpKNHh0bUVJMlllSTlSeS1lbmNUMDQ5Q2FEZ2VzWU03Y3JDSFM2UUtFZWZjanMyY3lfYnhIOWxGVkhHdUFGb1h0ejRlQXJId05vM1ZsaDdTMEY2VEV6LVlEU0RjUFlIdVV0R2E0bWw0?oc=5)

9. [브레인즈컴퍼니, +12.48% 상승폭 확대 - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxNSjM0Nk52MFoydUFqbk1PazJfWTZpWFoybDlQX2I0UmtqaFdXcUsxTE1xTXNmdXpHdnc1anU3V0VNM3NBcGhSSDJWT3dabHhUREtHdlAzRUlxdll3T2pPdGNHQ2lJeHRKN3VSVFlITUQ2RWYzR0plNFluTDRPeTJRVDJmNNIBlwFBVV95cUxQcXJaLWNXc0ZrX2tOZmExdzJpWEx3U05YbVZrbmVNSXdjN0h2M21FdlZZVXhER2o1OU1IMWtkVU9xTVJFbExGTWhQRVNlVmptc0RNcFFMYkhLRlR1WERuSzNsLUZlYUxlMXo3NzhWdml2eDZJUUZaTXRiZVVSSEk0R0JIZy1DY1E3c1VmUURLbEdlTGFTemow?oc=5)




최근 7일간 '이스트소프트(047560)' 관련 뉴스: (최대 10개):

1. [[강세 토픽] 딥페이크 테마, 알체라 +7.59%, 이스트소프트 +7.39% - 조선비즈 - 조선비즈](https://news.google.com/rss/articles/CBMigwFBVV95cUxOWWRCemdzVE9xY1JmWnVxM3pSLWl2RHhNbTZkbVlveXJUS2NHRW56M1NGRWpYWnVDMXJEODMzVWZ1X3J6djJVUDRnb3RJRS1Wd1BUUng1aTR4dVFTZFNhdDhjZDhNTEd4VkZyTXFEcl96LWVrVGdyUjVLNmRLQjM5Q3RsTdIBlwFBVV95cUxOSVdqeTVpMkhoZ2F6WHlOQmRuQ0R0V25NQnE2aXVOanlXX3VfWWNpakRQR19OYnpZRVBMNjFaOHlWMnpjeU9lZWNKQ3hiSVhvcHpTSXcyT3BYZ3dOQVdQUW9ST2hZZUl2N0RjOHpuTzJ2SGR6Y3AtQVZwWURrLXpUbHh5SGJndHVTLW5CLTBMTFlOYjVIb2lZ?oc=5)

2. [이스트소프트, 국제 NGO '더멋진세상'과 AI 교육 협력 - ZD넷 코리아](https://news.google.com/rss/articles/CBMiVkFVX3lxTE5hVjVTakRnSXZlY3VZOERHWlFldmtraVdObXFLaWFaWXIxWjZzUnpkUHd4ZnQzbWN2TXBNRnRKdlRIclBmd1BneXJJY2p2c1VQcjNmeFBn?oc=5)

3. [이스트소프트, '퍼플렉시티' 대항마 '앨런' 시장 침투율 확대 '주목' - 프라임경제](https://news.google.com/rss/articles/CBMibEFVX3lxTE9CbmZwbENmNFpoeE9TN1dEV0RXZmRVZzJpX0VjUnRUc0xyU0dJaWRGTk5OWlNnRmRxc2pqRUlETnBaOTFwWE5xdnIzZ3l6M05mSmY1OXltSDA5X3hoeE1EV1hyalJ4b09nU2VwSA?oc=5)

4. [이스트소프트, 국제구호개발 비정부기구 '더멋진세상'과 '페르소닷에이아이' 공급... 아프리카 청소년 IT 교육 위해 영상 오토 더빙 서비스 제공 - 인공지능신문](https://news.google.com/rss/articles/CBMiZ0FVX3lxTE9zNU83TFQxUzRVN1ZWMXJOS1dCUWpkMzYxSHJpcTFPeTBOaDZpbXI5UFlrM0Ytd1l1VEVTWENKazA4U2dyc3VscUx0UGlkcWVoY0hyYnpWU09lYVAzTkZEVFRpREdieDA?oc=5)

5. [[특징주]'AI 소프트웨어 기대' 이스트소프트, 3%대↑ - 아시아경제](https://news.google.com/rss/articles/CBMiX0FVX3lxTFBSTEoxTl84ajBTbWZuVnRRNkpnaFhHdTZmMkdKUU13Q3EtRy12WVdWejRKbFFLaEVZZUJVR3Z1emJWMFJBUjNJWkZBdzBKLWxKdkpJN2ZUSUtoTVBiM0hB?oc=5)

6. [[리포트 브리핑]이스트소프트, '신규 AI S/W 주목 받을 시점' Not Rated - 하나증권 - 뉴스핌](https://news.google.com/rss/articles/CBMiXEFVX3lxTE5ibnc3bTNlamZ6TVREblJsSlBhYWdqUVVfNUFfcEF4elo5YWF1blZCY1prOXk2Ni1kaUQxd3ZROGxMLS1ZWFRwaU5QaGZJbjVrZTZjdnVVRkNkMXIz0gFeQVVfeXFMTkRMd3BUQ3IzakFpcXY2alRNWnNPSVdkckFtU2Fxa0N0ZVhMbWZWQ2VoQkFtaDl6cm9XVl9Kc1NsTWY0bFFyTm9mT1dxWnpMY2tOTnNJUVlxcUNqbGN3dw?oc=5)

7. [이스트소프트, NGO 아프리카 청소년 IT 교육에 AI 비디오 트랜슬레이터 활용 - AI타임스](https://news.google.com/rss/articles/CBMiakFVX3lxTFBxMnFmREhEbWc4b1RDbU9FNVdLNjM4LUFVbEZWZ3RFQWRjVzVLWFB3Ukdja1RLeVFodHZ1NDZvQng3b2FPNmJ5OUtJMzMxRm8zVXViQmFNdDF4UE1leEEwWXlBMmc0c1FKY1E?oc=5)

8. [이스트소프트, ‘더멋진세상’과 아프리카 청소년 IT 교육에 AI 오토 더빙 제공 - 이데일리](https://news.google.com/rss/articles/CBMigAFBVV95cUxOOXBZRWR6NjR3THVqeWd0VHZlUDVSRFpuMk1UbEZRMk1kdXRxRzZKa1p1RHBBcUxNZXZqX2l1anJQaXJIeGxqMjV0NWFNLVFIYVdYOTN4OFBkT3hxbWtpaXJVVDA0UE91em01Q1drQnM0czJBYnBTY1M1cXgxYmJQQQ?oc=5)

9. [이스트소프트, NGO에 AI 휴먼 영상·오토 더빙 지원 - 전자신문](https://news.google.com/rss/articles/CBMiTkFVX3lxTE92dkhldVUtaDhiSkFZaTZqUnBZdlAwbXFQU094UGFlQjBLZDAzRVdkRVU2by1aYjhFOEg2Q1NHanZiZkFKOFZva0ItRndaQQ?oc=5)

10. [특징주, 이스트소프트-AI 챗봇(챗GPT 등) 테마 상승세에 5.98% ↑ - 매일경제](https://news.google.com/rss/articles/CBMiUkFVX3lxTFBHR2cwT2Z6bDFabzZzRTdKOWlPWGdFUmliQmRkRkJXa1NJSUZUcHIwRml5cjhPellRTGNFR01hSVRfZnBDTFhXNHBad2xIOERNTnc?oc=5)



In [7]:
# prompt: 엑셀에서 종목별 일자별 주가를 받아와서 보여준다

import pandas as pd

df = pd.read_excel("/content/종목일자별주가.xlsx", "Sheet1")

# 새로운 컬럼 추가 (초기값은 ' ')
for stock_name in df.columns:
    df[stock_name + '_Signal'] = ' '

# 80 이하이면 'T' 저장
for stock_name in df.columns:
    if stock_name.endswith('_Signal'):  # Signal 컬럼은 건너뛰기
        continue
    for index in df.index:
        if df.loc[index, stock_name] <= 80:
            df.loc[index, stock_name + '_Signal'] = 80

print(df)


    날짜      위메이드_0     RFHIC_1        야스_2        두산_3    코오롱티슈진_4  \
0    0  100.000000  100.000000  100.000000  100.000000  100.000000   
1    1   99.743590  100.069541   96.204934  100.485437   98.221950   
2    2   98.461538   97.566064   95.635674  101.456311   99.386879   
3    3   93.974359   83.796940   96.015180   97.330097   95.892091   
4    4  102.820513   89.986092   95.920304   99.029126  102.391171   
..  ..         ...         ...         ...         ...         ...   
56  56   97.948718   96.036161   76.660342  145.631068  174.126303   
57  57   97.820513   98.956885   79.127135  140.533981  182.096873   
58  58   98.461538   99.513213   82.542694  137.864078  175.352544   
59  59  102.435897  103.616134   80.170778  133.009709  175.659105   
60  60  102.564103  102.990264   79.601518  133.009709  184.855917   

    브릿지바이오테라퓨틱스_5     코스모화학_6     롯데케미칼_7     제이앤티씨_8  ...  코스모화학_6_Signal  \
0      100.000000  100.000000  100.000000  100.000000  ...                   
1  